# Notebook 03: Generalization

**Research Question:** Does action-grounding generalize across different tools?

This notebook:
1. Tests cross-tool transfer (escalate → search → sendMessage)
2. Computes 3×3 transfer matrix
3. Analyzes probe direction similarity
4. Visualizes activation space (t-SNE)

**Key hypothesis:** If transfer accuracy > 85%, representation is general, not tool-specific.

## Setup

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.utils.logging import setup_logging
from src.config import get_config
from src.data.io import load_activations
from src.analysis.probes import train_and_evaluate, load_probe, evaluate_probe
from src.analysis.visualization import plot_transfer_matrix

setup_logging(level="INFO")
config = get_config()

print("Generalization Analysis")

## 1. Load Activation Dataset

In [ ]:
# Load full activation dataset
dataset = load_activations(config.data.processed_dir / "activations.parquet")

print(f"Loaded {len(dataset)} activation samples")
print(f"\nTool type distribution:")
summary = dataset.summary()
print(summary['tools'])

In [ ]:
# Filter to single position and layer for consistency
# Use mid_response at layer 16
dataset_filtered = dataset.filter_by_position("mid_response").filter_by_layer(16)

print(f"\nFiltered dataset: {len(dataset_filtered)} samples")

## 2. Within-Tool Baseline

Train and test within each tool type to establish baseline.

In [ ]:
# Train within-tool probes
tool_types = ['escalate', 'search', 'sendMessage']
within_tool_accuracies = {}
tool_probes = {}

for tool in tool_types:
    print(f"\n{'='*60}")
    print(f"Training on {tool.upper()}")
    print(f"{'='*60}")
    
    tool_dataset = dataset_filtered.filter_by_tool(tool)
    
    if len(tool_dataset) < 50:
        print(f"  Not enough samples for {tool}: {len(tool_dataset)}")
        continue
    
    probe, train_metrics, test_metrics = train_and_evaluate(
        tool_dataset,
        label_type="reality",
        test_size=0.2,
        random_state=42,
    )
    
    within_tool_accuracies[tool] = test_metrics.accuracy
    tool_probes[tool] = probe
    
    print(f"\n  Within-tool accuracy: {test_metrics.accuracy:.1%}")

print("\nWithin-tool accuracies:")
for tool, acc in within_tool_accuracies.items():
    print(f"  {tool}: {acc:.1%}")

## 3. Cross-Tool Transfer Matrix

**Key experiment:** Train on one tool, test on another.

In [ ]:
# Compute full transfer matrix
transfer_matrix = np.zeros((len(tool_types), len(tool_types)))

for i, train_tool in enumerate(tool_types):
    for j, test_tool in enumerate(tool_types):
        print(f"\nTrain on {train_tool}, test on {test_tool}...")
        
        # Get datasets
        train_dataset = dataset_filtered.filter_by_tool(train_tool)
        test_dataset = dataset_filtered.filter_by_tool(test_tool)
        
        if len(train_dataset) < 50 or len(test_dataset) < 50:
            print(f"  Insufficient data")
            continue
        
        # Train on train_tool
        X_train, y_train = train_dataset.to_sklearn_format("reality")
        X_test, y_test = test_dataset.to_sklearn_format("reality")
        
        from src.analysis.probes import train_probe
        probe, _, _ = train_probe(X_train, y_train, random_state=42)
        
        # Test on test_tool
        test_metrics = evaluate_probe(probe, X_test, y_test)
        
        transfer_matrix[i, j] = test_metrics.accuracy
        
        transfer_type = "Within-tool" if i == j else "Cross-tool"
        print(f"  {transfer_type} accuracy: {test_metrics.accuracy:.1%}")

print("\nTransfer Matrix:")
print(pd.DataFrame(transfer_matrix, index=tool_types, columns=tool_types))

In [ ]:
# Visualize transfer matrix
fig = plot_transfer_matrix(
    transfer_matrix,
    tool_labels=[t.capitalize() for t in tool_types],
    title="Cross-Tool Transfer Accuracy",
    save_path=config.figures_dir / "figure4_transfer_matrix",
)

plt.show()

In [ ]:
# Compute average cross-tool transfer
# Exclude diagonal (within-tool)
mask = ~np.eye(len(tool_types), dtype=bool)
cross_tool_accuracies = transfer_matrix[mask]
mean_cross_tool = np.mean(cross_tool_accuracies)
std_cross_tool = np.std(cross_tool_accuracies)

mean_within_tool = np.mean(np.diag(transfer_matrix))

print(f"\n**Transfer Statistics:**")
print(f"  Mean within-tool accuracy: {mean_within_tool:.1%}")
print(f"  Mean cross-tool accuracy: {mean_cross_tool:.1%} ± {std_cross_tool:.3f}")
print(f"  Accuracy drop: {(mean_within_tool - mean_cross_tool):.1%}")

if mean_cross_tool > 0.85:
    print("\n  ✓ Cross-tool accuracy > 85%")
    print("  → Representation generalizes across tools!")
else:
    print(f"\n  ✗ Cross-tool accuracy = {mean_cross_tool:.1%} < 85%")
    print("  → Generalization may be limited")

## 4. t-SNE Visualization

Visualize activation space colored by tool type.

In [ ]:
# Sample for t-SNE (full dataset too large)
from sklearn.manifold import TSNE

# Sample 500 points (balanced across tools)
sampled_indices = []
for tool in tool_types:
    tool_indices = [i for i, s in enumerate(dataset_filtered.samples) if s.tool_type == tool]
    if len(tool_indices) > 166:
        np.random.seed(42)
        tool_indices = np.random.choice(tool_indices, 166, replace=False)
    sampled_indices.extend(tool_indices)

X_sample = dataset_filtered.activations[sampled_indices]
tool_labels = [dataset_filtered.samples[i].tool_type for i in sampled_indices]
action_labels = [dataset_filtered.samples[i].tool_used for i in sampled_indices]

print(f"Running t-SNE on {len(X_sample)} samples...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_sample)

print("t-SNE complete.")

In [ ]:
# Plot t-SNE colored by tool type
fig, ax = plt.subplots(figsize=(10, 8))

colors = {'escalate': 'red', 'search': 'blue', 'sendMessage': 'green'}

for tool in tool_types:
    mask = np.array([t == tool for t in tool_labels])
    ax.scatter(
        X_tsne[mask, 0],
        X_tsne[mask, 1],
        c=colors[tool],
        label=tool.capitalize(),
        alpha=0.6,
        s=50,
    )

ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_title('Activation Space Colored by Tool Type')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(config.figures_dir / "tsne_by_tool.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot t-SNE colored by action (tool_used)
fig, ax = plt.subplots(figsize=(10, 8))

action_colors = {True: 'green', False: 'red'}
action_names = {True: 'Tool Used', False: 'No Tool'}

for action in [True, False]:
    mask = np.array([a == action for a in action_labels])
    ax.scatter(
        X_tsne[mask, 0],
        X_tsne[mask, 1],
        c=action_colors[action],
        label=action_names[action],
        alpha=0.6,
        s=50,
    )

ax.set_xlabel('t-SNE 1')
ax.set_ylabel('t-SNE 2')
ax.set_title('Activation Space Colored by Tool Usage (Ground Truth)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(config.figures_dir / "tsne_by_action.png", dpi=300, bbox_inches='tight')
plt.show()

## 5. Statistical Significance

Is cross-tool transfer significantly above chance?

In [ ]:
# Permutation test on cross-tool transfer
from scipy.stats import ttest_1samp

# Test if cross-tool accuracies > 0.5 (chance)
t_stat, p_value = ttest_1samp(cross_tool_accuracies, 0.5)

print(f"\nOne-sample t-test (H0: accuracy = 0.5):")
print(f"  t = {t_stat:.3f}")
print(f"  p = {p_value:.4e}")
print(f"  Significant: {p_value < 0.05}")

# Effect size
from src.analysis.statistics import compute_effect_size
cohen_d = (mean_cross_tool - 0.5) / std_cross_tool
print(f"  Cohen's d: {cohen_d:.3f}")

## Summary

In [ ]:
print("=" * 60)
print("PHASE 3 RESULTS: GENERALIZATION")
print("=" * 60)

print(f"\nWithin-Tool Accuracy:")
for tool, acc in within_tool_accuracies.items():
    print(f"  {tool}: {acc:.1%}")
print(f"  Mean: {mean_within_tool:.1%}")

print(f"\nCross-Tool Transfer:")
print(f"  Mean accuracy: {mean_cross_tool:.1%} ± {std_cross_tool:.3f}")
print(f"  Accuracy drop: {(mean_within_tool - mean_cross_tool):.1%}")

print(f"\nStatistical Significance:")
print(f"  p-value: {p_value:.4e}")
print(f"  Effect size (Cohen's d): {cohen_d:.3f}")

if mean_cross_tool > 0.85:
    print("\n✓ Cross-tool transfer > 85%")
    print("✓ Representation is general, not tool-specific")
else:
    print(f"\n⚠ Cross-tool transfer = {mean_cross_tool:.1%}")
    print("⚠ Generalization limited (but still above chance)")

print("=" * 60)

## Next Steps

→ **Notebook 04:** Causal intervention via steering vectors